In [7]:
import requests
import pandas as pd

# Using key
api_key = "My key"
latitude = 39.7684
longitude = -86.1581
keyword = "food trucks"

base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
details_url = "https://maps.googleapis.com/maps/api/place/details/json"

#List for found food trucks
food_trucks = []

#Main method to search for food trucks and all necessary information about them
def get_data(page_token=None):
    params = {
        "query": keyword,
        "location": f"{latitude},{longitude}",
        "key": api_key,
        "pagetoken": page_token,
    }
    response = requests.get(base_url, params=params)
    data = response.json()

    for result in data.get("results", []):
        place_id = result["place_id"]
        details_params = {
            "place_id": place_id,
            "key": api_key,
        }
        details_response = requests.get(details_url, params=details_params)
        details_data = details_response.json()

        name = result["name"]
        address = details_data["result"]["formatted_address"]
        website = details_data["result"].get("website", "")
        open_hours = details_data["result"].get("opening_hours", {}).get("weekday_text", "")
        
        # Get latitude and longitude
        location = details_data["result"]["geometry"]["location"]
        lat = location["lat"]
        lng = location["lng"]
        
        #Adding Name, address, Website, Hours, Lat, Long
        food_trucks.append({
            "Name": name,
            "Address": address,
            "Website": website,
            "Open Hours": "\n".join(open_hours),
            "Latitude": lat,
            "Longitude": lng
        })

    next_token = data.get("next_page_token")
    return next_token

next_token = get_data()
while next_token:
    next_token = get_data(next_token)

#creating dataframe out of the list
food_trucks_df = pd.DataFrame(food_trucks)

# Save the data to a CSV file
food_trucks_df.to_csv("food_trucks.csv", index=False)

print(food_trucks_df)


                                              Name  \
0                                     ClusterTruck   
1                         The Night Owl Food Truck   
2                         Tortas Guicho Food Truck   
3                         Pico de Gallo Food Truck   
4                                    T Street Eatz   
5                     El Amigo Taqueria Food Truck   
6                             El Viejon Food Truck   
7                                     Tha Corn Man   
8                                Taqueria Mi Costa   
9                            Food truck Mil Amores   
10                          Super Tacos food truck   
11                  Antojitos Guzman food truck #1   
12                        El Buenavista Food Truck   
13                               KG Slider Station   
14                   Mi Lindo Pénjamo - Food Truck   
15                   The Latin Flavor - Food Truck   
16                                Fiesta on The Go   
17                       big

In [15]:
import pandas as pd
import folium
from folium import plugins
import geopy.distance

# Load the data from the CSV file with coordinates
food_trucks_df = pd.read_csv("food_trucks.csv")

# Create a two-day foodie plan
day1 = food_trucks_df.head(3)
day2 = food_trucks_df.iloc[3:6]

# Combine the food trucks for both days
foodie_plan = pd.concat([day1, day2], ignore_index=True)

# Add a "Time" column to the foodie plan
foodie_plan["Time"] = ["10:00 AM", "12:00 PM", "2:00 PM"] * 2

# Calculate travel distance and time using the Haversine formula
start_location = (39.7684, -86.1581)  # Starting location

def calculate_distance_and_time(row):
    destination = (row['Latitude'], row['Longitude'])
    distance = geopy.distance.distance(start_location, destination).miles
    time = distance / 30.0 * 60  # Convert travel time to minutes (assuming an average driving speed of 30 miles per hour)
    return round(distance, 2), round(time, 2)

# Calculate distances and times
distances, times = zip(*foodie_plan.apply(calculate_distance_and_time, axis=1))
foodie_plan['Travel Distance (miles)'] = distances
foodie_plan['Travel Time (minutes)'] = times

# Set "Transportation Type" based on travel time
foodie_plan['Transportation Type'] = foodie_plan.apply(
    lambda row: 'Walk' if row['Travel Time (minutes)'] < 5 else 'Car',
    axis=1
)

# Drop the 'Latitude' and 'Longitude' columns
foodie_plan.drop(['Latitude', 'Longitude'], axis=1, inplace=True)

# Save the foodie plan to a CSV file
foodie_plan.to_csv("foodie_plan.csv", index=False)

# Create a map to visualize the food truck locations and routes
map_center = [39.7684, -86.1581]
foodie_map = folium.Map(location=map_center, zoom_start=13)

# Add markers for food truck locations
for index, row in foodie_plan.iterrows():
    folium.Marker(
        location=[food_trucks_df.loc[index, 'Latitude'], food_trucks_df.loc[index, 'Longitude']],
        popup=row['Name'],
    ).add_to(foodie_map)

# Create a travel route using coordinates
route_points = [(39.7684, -86.1581)]  # Starting point
route_points += [(food_trucks_df.loc[index, 'Latitude'], food_trucks_df.loc[index, 'Longitude']) for index, row in foodie_plan.iterrows()]
folium.PolyLine(route_points, color="blue").add_to(foodie_map)

# Save the map to an HTML file
foodie_map.save("foodie_route.html")

# Print the updated foodie plan with details
print(foodie_plan)


                           Name  \
0                  ClusterTruck   
1      The Night Owl Food Truck   
2      Tortas Guicho Food Truck   
3      Pico de Gallo Food Truck   
4                 T Street Eatz   
5  El Amigo Taqueria Food Truck   

                                             Address  \
0  729 N Pennsylvania St, Indianapolis, IN 46204,...   
1  South and, S Meridian St, Indianapolis, IN 462...   
2  S. East St and, Fletcher Ave, Indianapolis, IN...   
3  3014 Lafayette Rd #3000, Indianapolis, IN 4622...   
4  City Market, 222 E Market St, Indianapolis, IN...   
5        3356 W 16th St, Indianapolis, IN 46222, USA   

                                             Website  \
0  https://downtown-indy.clustertruck.com/menu?ut...   
1                        http://mercersnightowl.com/   
2                                                NaN   
3            https://www.facebook.com/peregrino1982/   
4                    http://www.tstreeteatzindy.com/   
5                        